In [1]:
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
import requests
import time
import threading
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the scaler and model
scaler = joblib.load('scaler.pkl')
model = load_model('fetal_movement_detection_model.h5')

In [3]:
# Define a function to normalize and create sequences
def preprocess_data(data_buffer, seq_length=10):
    # Normalize data
    normalized_data = scaler.transform(data_buffer)
    
    # Create sequences
    sequences = []
    for i in range(len(normalized_data) - seq_length + 1):
        sequence = normalized_data[i:i + seq_length]
        sequences.append(sequence)
    return np.array(sequences)

In [4]:
# Function to collect real-time data from ThingSpeak
def collect_real_time_data(api_key, channel_id, fields):
    url = f'https://api.thingspeak.com/channels/{channel_id}/feeds.json?api_key={api_key}&results=1'
    response = requests.get(url)
    data = response.json()
    if 'feeds' in data and len(data['feeds']) > 0:
        latest_feed = data['feeds'][0]
        return np.array([[float(latest_feed[field]) for field in fields]])
    else:
        return None

In [5]:
# ThingSpeak configuration
api_key = 'ETT7ULQV6FR64XWA'
channel_id = '2538997'
fields = ['field1', 'field2', 'field3', 'field4', 'field5', 'field6']  # Update fields if necessary

In [6]:
# Buffer to store incoming data
data_buffer = []

In [7]:
# Sequence length required by the model
seq_length = 10

In [8]:
# Flag to control the loop
stop_flag = False

In [9]:
# Function to handle real-time data collection and prediction
def real_time_prediction():
    global stop_flag
    while not stop_flag:
        # Collect data in real-time
        new_data = collect_real_time_data(api_key, channel_id, fields)
        if new_data is not None:
            data_buffer.append(new_data)
        
            # Keep the buffer size fixed
            if len(data_buffer) > seq_length:
                data_buffer.pop(0)
            
            # Process data if we have enough for one sequence
            if len(data_buffer) == seq_length:
                sequences = preprocess_data(np.array(data_buffer).reshape(-1, 6), seq_length)
                
                # Make predictions
                predictions = model.predict(sequences)
                
                # Interpret predictions (assuming softmax output)
                predicted_class = np.argmax(predictions, axis=-1)
                
                # Map predicted class to activity label
                activity_labels = {0: 'running', 1: 'walking', 2: 'other', 3: 'sleeping'}
                predicted_activity = [activity_labels[label] for label in predicted_class]
                
                # Print or act on the prediction
                print(f"Predicted activity: {predicted_activity[-1]}")
        
        # Add some delay to simulate real-time data collection
        time.sleep(1)  # Set appropriate delay for your data collection rate

In [10]:
# Function to stop the prediction loop
def stop_prediction():
    global stop_flag
    input("Press Enter to stop...\n")
    stop_flag = True

In [11]:
# Start the real-time prediction in a separate thread
prediction_thread = threading.Thread(target=real_time_prediction)
prediction_thread.start()

C:\Users\saman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step
Predicted activity: walking
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


C:\Users\saman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Predicted activity: walking
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


C:\Users\saman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

Predicted activity: walking
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


C:\Users\saman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Predicted activity: walking
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


C:\Users\saman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Predicted activity: walking


In [ ]:
# Start the stop prediction function in the main thread
stop_prediction()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


C:\Users\saman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Predicted activity: walking
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step


C:\Users\saman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Predicted activity: walking
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


...

Predicted activity: walking


C:\Users\saman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Predicted activity: walking
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


C:\Users\saman\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Predicted activity: walking


In [14]:
# Wait for the prediction thread to finish
prediction_thread.join()

print("Real-time prediction stopped.")

Real-time prediction stopped.
